In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

Base Code (cleaned up version)

In [19]:
def initialize_points(num_points):
    # Initialize points list and counters
    points = []
    point_counter = 0

    def onMouse(event, x, y, flags, params):
        nonlocal point_counter
        if event == cv2.EVENT_LBUTTONDOWN:
            points.append((x, y))
            point_counter += 1
            if point_counter == num_points:
                cv2.destroyWindow("Camera")  # Close the camera window after selecting all points

    cv2.namedWindow("Camera")
    cv2.setMouseCallback("Camera", onMouse)

    cap = cv2.VideoCapture(0)
    old_frame = None  # Initialize old_frame

    while point_counter < num_points:
        _, frame = cap.read()

        # Display instructions for selecting points
        font_scale = 0.7
        font_color = (0, 225, 0)
        font_thickness = 2
        cv2.putText(frame, f"Select {num_points} points to track (left-click)", (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                    font_scale, font_color, font_thickness)

        cv2.imshow("Camera", frame)

        key = cv2.waitKey(1)
        if key == 27:  # ESC key to exit
            break

    old_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    old_points = np.array(points, dtype="float32").reshape(-1, 1, 2)
    cap.release()
    cv2.destroyAllWindows()

    return old_frame, old_points, points

def track_points(old_frame, old_points, num_points, points):
    cap = cv2.VideoCapture(0)

    while True:
        _, frame2 = cap.read()

        if frame2 is None:  # Check if frame2 is valid
            break

        new_frame = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

        new_points, status, error = cv2.calcOpticalFlowPyrLK(old_frame, new_frame, old_points,
                                                             None, maxLevel=1,
                                                             criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,
                                                                       15, 0.08))

        frame2_copy = frame2.copy()  # Create a copy for displaying points

        # Draw slightly bigger dots at the new points
        for i in range(num_points):
            if status.ravel()[i] == 1:  # Check if the point is successfully tracked
                x, y = new_points[i].ravel()
                mask = np.zeros_like(frame2_copy)
                cv2.circle(mask, (int(x), int(y)), 5, (0, 255, 0), -1)  # Increase the radius to make it bigger
                frame2_copy = cv2.addWeighted(frame2_copy, 0.7, mask, 0.3, 0.1)

                # Display coordinates of the selected point and tracking result
                font_scale = 0.4
                font_color = (0, 225, 0)
                font_thickness = 1
                cv2.putText(frame2_copy, f"Point {i + 1} Coordinates: ({int(x)}, {int(y)})", (10, 30 + i * 30),
                            cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_color, font_thickness)

        cv2.imshow("Camera", frame2_copy)

        old_frame = new_frame.copy()
        old_points = new_points.copy()

        key = cv2.waitKey(1)
        if key == 27:  # ESC key to exit
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    num_points = 3
    old_frame, old_points, points = initialize_points(num_points)
    print("Selected Point Coordinates: ", points)
    track_points(old_frame, old_points, num_points, points)

Selected Point Coordinates:  [(227, 208), (429, 242), (246, 436)]


Three Points Tracked

In [21]:
def initialize_points(num_points):
    # Initialize points list and counters
    points = []
    point_counter = 0

    def onMouse(event, x, y, flags, params):
        nonlocal point_counter
        if event == cv2.EVENT_LBUTTONDOWN:
            points.append((x, y))
            point_counter += 1
            if point_counter == num_points:
                cv2.destroyWindow("Camera")  # Close the camera window after selecting all points

    cv2.namedWindow("Camera")
    cv2.setMouseCallback("Camera", onMouse)

    cap = cv2.VideoCapture(0)
    old_frame = None  # Initialize old_frame

    while point_counter < num_points:
        _, frame = cap.read()

        # Display instructions for selecting points
        font_scale = 0.7
        font_color = (0, 225, 0)
        font_thickness = 2
        cv2.putText(frame, f"Select {num_points} points to track (left-click)", (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                    font_scale, font_color, font_thickness)

        # Draw circles on selected points
        for point in points:
            cv2.circle(frame, point, 5, (0, 0, 255), -1)  # Draw a red circle on each selected point

        cv2.imshow("Camera", frame)

        key = cv2.waitKey(1)
        if key == 27:  # ESC key to exit
            break

    old_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    old_points = np.array(points, dtype="float32").reshape(-1, 1, 2)
    cap.release()
    cv2.destroyAllWindows()

    return old_frame, old_points, points

def track_points(old_frame, old_points, num_points, points):
    cap = cv2.VideoCapture(0)

    while True:
        _, frame2 = cap.read()

        if frame2 is None:  # Check if frame2 is valid
            break

        new_frame = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

        new_points, status, error = cv2.calcOpticalFlowPyrLK(old_frame, new_frame, old_points,
                                                             None, maxLevel=1,
                                                             criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,
                                                                       15, 0.08))

        # Draw slightly bigger dots at the new points
        for i in range(num_points):
            if status.ravel()[i] == 1:  # Check if the point is successfully tracked
                x, y = new_points[i].ravel()
                mask = np.zeros_like(frame2)
                cv2.circle(mask, (int(x), int(y)), 5, (0, 255, 0), -1)  # Increase the radius to make it bigger
                frame2 = cv2.addWeighted(frame2, 0.7, mask, 0.3, 0.1)

                # Display coordinates of the selected point and tracking result
                font_scale = 0.7
                font_color = (0, 225, 0)
                font_thickness = 1
                cv2.putText(frame2, f"Point {i + 1} Coordinates: ({points[i][0]}, {points[i][1]})", (10, 30 + i * 30),
                            cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_color, font_thickness)

        cv2.imshow("Camera", frame2)

        old_frame = new_frame.copy()
        old_points = new_points.copy()

        key = cv2.waitKey(1)
        if key == 27:  # ESC key to exit
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    num_points = 3
    old_frame, old_points, points = initialize_points(num_points)
    print("Selected Point Coordinates: ", points)
    track_points(old_frame, old_points, num_points, points)

Selected Point Coordinates:  [(276, 214), (454, 243), (285, 430)]


Integration of Lucas-Kanade Optical Flow Tracking using recorded Videos